# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=guillaume password=test")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
song_files

['/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACQT128F9331780.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACER128F4290F96.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACZK128F4243829.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACCG128F92E8A55.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACOW128F933E35F.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACHN128F1489601.json',
 '/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAA

In [7]:
song_files[0]

'/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/song_data/A/A/C/TRAACQT128F9331780.json'

In [8]:
df = pd.read_json(song_files[0], typ='series')
df = df.to_frame().T

In [9]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR1Y2PT1187FB5B9CE,27.94017,-82.32547,Brandon,John Wesley,SOLLHMX12AB01846DC,The Emperor Falls,484.62322,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df.loc[0,['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.values.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df.loc[0,['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_data.values.tolist()

In [13]:
artist_data

['AR1Y2PT1187FB5B9CE', 'John Wesley', 'Brandon', 27.94017, -82.32547]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
artist_table_insert

'\nINSERT INTO artists (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)\nVALUES (%s, %s, %s, %s, %s)\nON CONFLICT (artist_id)\nDO NOTHING\n'

In [15]:
try: 
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')

In [17]:
# log_files

In [18]:
filepath = log_files[0]

In [19]:
filepath

'/media/documents/data/udacity/data_engineering/data_modelisation/projects/data_modeling_postgres/data/log_data/2018/11/2018-11-13-events.json'

In [20]:
df = pd.read_json(filepath, lines=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         339 non-null    object 
 1   auth           394 non-null    object 
 2   firstName      383 non-null    object 
 3   gender         383 non-null    object 
 4   itemInSession  394 non-null    int64  
 5   lastName       383 non-null    object 
 6   length         339 non-null    float64
 7   level          394 non-null    object 
 8   location       383 non-null    object 
 9   method         394 non-null    object 
 10  page           394 non-null    object 
 11  registration   383 non-null    float64
 12  sessionId      394 non-null    int64  
 13  song           339 non-null    object 
 14  status         394 non-null    int64  
 15  ts             394 non-null    int64  
 16  userAgent      383 non-null    object 
 17  userId         394 non-null    object 
dtypes: float64

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
# df.page == 'NextSong'

In [23]:
df = df[df.page == 'NextSong']
# df.info()

In [24]:
# df.head()

In [25]:
df['datetime'] = pd.to_datetime(df['ts'], unit='ms')

In [26]:
t = pd.to_datetime(df['ts'])

In [27]:
# df['datetime'].dt.date

In [28]:
# dir(df['datetime'].dt)

In [29]:
time_data = []
time_data.extend((df['ts'],
                       df['datetime'].dt.hour,
                       df['datetime'].dt.day,
                       df['datetime'].dt.isocalendar().week,
                       df['datetime'].dt.month,
                       df['datetime'].dt.year,
                       df['datetime'].dt.weekday))

In [30]:
column_labels = ['timestamp','hour', 'day', 'week', 'month', 'year', 'weekday']

In [31]:
time_dict = {}
for k, v in zip(column_labels, time_data):
    time_dict[k] = v

In [32]:
time_df = pd.DataFrame(time_dict)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
1,1542069637796,0,13,46,11,2018,1
3,1542071549796,1,13,46,11,2018,1
4,1542079142796,3,13,46,11,2018,1
5,1542081112796,3,13,46,11,2018,1
8,1542085206796,5,13,46,11,2018,1


In [33]:
# for i in time_df.iterrows():
#     print(i)
#     break

In [34]:
# list(time_df.iterrows())[0]

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [35]:
time_table_insert

'\nINSERT INTO time (start_time, hour, day, week, month, year, weekday)\nVALUES (%s, %s, %s, %s, %s, %s, %s)\n'

In [36]:
try:
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [37]:
df.sample(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime
210,Josh Groban,Logged In,Kate,F,15,Harrell,244.45342,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,537,Petit Papa Noel (Album Version),200,1542129654796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-13 17:20:54.796
389,22-20s,Logged In,Sara,F,67,Johnson,237.16526,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Such A Fool,200,1542152631796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,2018-11-13 23:43:51.796


In [38]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']
user_df = user_df.drop_duplicates()
user_df.sample(5)

,user_id,first_name,last_name,gender,level
65,6,Cecilia,Owens,F,free
4,9,Wyatt,Scott,M,free
355,55,Martin,Johnson,M,free
270,95,Sara,Johnson,F,paid
106,14,Theodore,Harris,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [39]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [40]:
# columns : artist, length, song

In [41]:
# try:
#     for index, row in df.iterrows():
#         # get songid and artistid from song and artist tables
#         cur.execute(song_select, (row.song, row.artist, row.length))
#         results = cur.fetchone()
#         print(results)
# except psycopg2.Error as e: 
#     print("Error: Inserting Rows")
#     print (e)

In [42]:
for index, row in df.iterrows():
    print(index)
    print(row)
    break

1
artist                                                          Fu
auth                                                     Logged In
firstName                                                    Kevin
gender                                                           M
itemInSession                                                    1
lastName                                                  Arellano
length                                                   280.05832
level                                                         free
location                                   Harrisburg-Carlisle, PA
method                                                         PUT
page                                                      NextSong
registration                                       1540006905796.0
sessionId                                                      514
song                                                       Ja I Ty
status                                                      

In [43]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(row.song)

Ja I Ty
A Party Song (The Walk of Shame)
Pop-Pop!
C'mon N' Ride It (The Train) (LP Version)
Born To Lead
The Pretender
Side .002
Too Late For Goodbyes
Sister Twisted
Ever So Sweet
9 Crimes (Demo)
Crying Blue Rain
Peito Vazio
Lady D'Arbanville
You're The One
Do You Hear The People Sing?
Work To Do
When A Woman Cries
Till The Sky Falls Down
Sealed With A Kiss
Your Touch
Circles (2007 Live At The Moore Theater in Seattle LP Version)
Howlin For You
This Is Your Life
Fingers And Thumbs (Cold Summer's Day) (Sound Factory Remix)
Snow [Hey Oh] (Album Version)
Bulgarian  Chicks
Tiger Mountain Peasant Song
Man With Bindle [Pawt 1] (2004)
Sehr kosmisch
Over And Over
Break The Night With Colour
Chemistry
The Sun In The Stream
Mockingbird
Love Walks In (Remastered Album Version)
Never Alone (Acoustic Version)
Way Out of Here [Album Version]
Beautiful Dangerous (featuring Fergie)
Alejandro
Never Ever
Who I Am
Pequenos Olhos
The Fear You Won't Fall
Revelry
Rock 'n' Roll Creation
Beside You In Time
B

In [81]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent 
    songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [46]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.